In [1]:
import torch
import torchvision
from torch.nn import functional as F
import torchvision.transforms.functional as transFunc
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

### Simple CNN with no regualarization

In [2]:
class CNN_Clf(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
        
        
    def forward(self, x):
        out = F.max_pool2d(F.relu(self.conv1(x)), 2)
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)
        out = out.view(-1, 32*8*8)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        
        return out

In [3]:
CNN_Clf()

CNN_Clf(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

### CNN with Dropout

**Dropout:** what if we just don’t train a random bunch of nodes within the network during a training cycle? Because they won’t be updated, they won’t have the chance to overfit to the input data, and because it’s random, each training cycle will ignore a different selection of the input, which should help generalization even further.

In [4]:
class CNN_Clf_do(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
        self.dropout = nn.Dropout2d(p=0.3)
        
    def forward(self, x):
        out = F.max_pool2d(F.relu(self.conv1(x)), 2)
        out = F.max_pool2d(F.relu(self.conv2(out)), 2)
        out = out.view(-1, 32*8*8)
        out = self.dropout(F.relu(self.fc1(out)))
        out = self.fc2(out)
        
        return out

In [5]:
CNN_Clf_do()

CNN_Clf_do(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout2d(p=0.3, inplace=False)
)

### CNN with BatchNorm

BatchNorm, short for batch normalization, is a simple layer that has one task in life: using two learned parameters (meaning that it will be trained along with the rest of the network) to try to ensure that each minibatch that goes through the network has a mean centered around zero with a variance of 1. You might ask why we need to do this when we’ve already normalized our input by using the transform chain in Chapter 2. For smaller networks, BatchNorm is indeed less useful, but as they get larger, the effect of any layer on another, say 20 layers down, can be vast because of repeated multiplication, and you may end up with either vanishing or exploding gradients, both of which are fatal to the training process. The BatchNorm layers make sure that even if you use a model such as ResNet-152, the multiplications inside your network don’t get out of hand.

You might be wondering: if we have BatchNorm in our network, why are we normalizing the input at all in the training loop’s transformation chain? After all, shouldn’t BatchNorm do the work for us? And the answer here is yes, you could do that! But it’ll take longer for the network to learn how to get the inputs under control, as they’ll have to discover the initial transform themselves, which will make training longer.

In [33]:
class CNN_Clf_BN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=32)       
        self.fc1 = nn.Linear(32*8*8, 32)
        self.bn3 = nn.BatchNorm1d(num_features=32) 
        self.fc2 = nn.Linear(32, 2)
        
        
    def forward(self, x):
        out = F.max_pool2d(F.relu(self.bn1(self.conv1(x))), 2)
        out = F.max_pool2d(F.relu(self.bn2(self.conv2(out))), 2)
        out = out.view(-1, 32*8*8)
        out = F.relu(self.fc1(out))
        out = self.fc2(self.bn3(out))
        
        return out

In [34]:
CNN_Clf_BN()

CNN_Clf_BN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=2048, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

In [7]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='/home/mayur/Desktop/Pytorch/data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified


In [8]:
testset = torchvision.datasets.CIFAR10(root='/home/mayur/Desktop/Pytorch/data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


In [9]:
#Label Mapping - classification of two class 

#CIFAR has 10 classes, we are restricting it into 2 classes
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar_2 = [(img, label_map[label1]) for img, label1 in trainset if label1 in [0,2]]
cifar2_val = [(img, label_map[label]) for img, label in testset if label in [0, 2]]

trainloader = torch.utils.data.DataLoader(cifar_2, batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,shuffle=False)

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [11]:
def train(model, optimizer, loss_fn, trainloader, testloader, n_epochs, device):
    for epoch in range(1, n_epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch in trainloader:
            img, label = batch
            img = img.to(device)
            label = label.to(device)
            
            output = model(img)
            optimizer.zero_grad()
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()
        training_loss /= len(trainset)
        
        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in testloader:
            img, label = batch
            img = img.to(device)
            label = label.to(device)
            
            output = model(img)
            loss = loss_fn(output, label)
            valid_loss += loss.data.item()
            correct = torch.eq(torch.max(F.softmax(output),dim=1)[1], label).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
            
        valid_loss/=len(testset)
        
        print(f'Epoch: {epoch}, Training Loss: {training_loss:.3f}, Validation Loss: {valid_loss:.3f}, Accuracy: {num_correct/num_examples}')
            

In [12]:
### CNN
net = CNN_Clf()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.to(device)
train(net, optimizer, torch.nn.CrossEntropyLoss(), trainloader, testloader, n_epochs=20, device=device)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.832
Epoch: 2, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8695
Epoch: 3, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.867
Epoch: 4, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8925
Epoch: 5, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.9035
Epoch: 6, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.905
Epoch: 7, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.903
Epoch: 8, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8905
Epoch: 9, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.9065
Epoch: 10, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.876
Epoch: 11, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.9035
Epoch: 12, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.898
Epoch: 13, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.905
Epoch: 14, Training Loss: 0.000, Validation Loss: 0.00

In [13]:
### CNN
net = CNN_Clf_do()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.to(device)
train(net, optimizer, torch.nn.CrossEntropyLoss(), trainloader, testloader, n_epochs=20, device=device)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.862
Epoch: 2, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8665
Epoch: 3, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8825
Epoch: 4, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.892
Epoch: 5, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8865
Epoch: 6, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.872
Epoch: 7, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.899
Epoch: 8, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.9055
Epoch: 9, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.911
Epoch: 10, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.9025
Epoch: 11, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.91
Epoch: 12, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.912
Epoch: 13, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.909
Epoch: 14, Training Loss: 0.000, Validation Loss: 0.001,

In [35]:
### CNN
net = CNN_Clf_BN()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.to(device)
train(net, optimizer, torch.nn.CrossEntropyLoss(), trainloader, testloader, n_epochs=20, device=device)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.868
Epoch: 2, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.834
Epoch: 3, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.871
Epoch: 4, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8905
Epoch: 5, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8875
Epoch: 6, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.8935
Epoch: 7, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.9045
Epoch: 8, Training Loss: 0.001, Validation Loss: 0.001, Accuracy: 0.9055
Epoch: 9, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.8995
Epoch: 10, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.889
Epoch: 11, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.907
Epoch: 12, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.892
Epoch: 13, Training Loss: 0.000, Validation Loss: 0.001, Accuracy: 0.9015
Epoch: 14, Training Loss: 0.000, Validation Loss: 0.0